Merry Christmas everyone! 

[Advent of Code day 3](http://adventofcode.com/2018/day/3)

## Part 1

Slightly more complicated from here. 

We'll want regexps for the input, and arrays for the claims

In [1]:
import re

import numpy as np

In [2]:
m=re.match('(.+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)', '#1 @ 1,3: 4x4')

OK... now I'd like a function which takes a sheet size, offset and size and returns an `np.ndarray` of zeros, except for the bit that's claimed.

In [3]:
def claim(sheet_size, offset_x, offset_y, size_x, size_y):
    '''
    Return an ndarray of size sheet_size square, with
    zeros except for the claim (which is 1)
    '''
    # Set up the initial sheet
    out=np.zeros((sheet_size, sheet_size), dtype=np.int)
    
    # set the values of the claimed square to 1
    # Probably a better way than with a for loop,
    # but I don't know numpy that well
    for i in range(offset_y, offset_y+size_y):
        out[i][offset_x:offset_x+size_x]=1
    
    
    return out

In [4]:
a=claim(10, 3, 2, 5, 4)

a

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Seems OK. 

Now, want to create one of these arrays for each claim, and then sum them. Let's parse the input, and use the list of claims to ascertain how big the original sheet needs to be:

In [5]:
def parse_input(string_in):
    '''
    Return a dict of claims, indexed by the claim ID
    '''
    out_dict={}
    for nl in string_in.split('\n'):
        m=re.match('(.+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)', nl.strip())
        if m:
            out_dict[m.group(1)]=[int(m.group(x)) for x in range(2, 6)]
    return out_dict

In [6]:
d=parse_input('''
    #1 @ 1,3: 4x4
    #2 @ 3,1: 4x4
    #3 @ 5,5: 2x2
    ''')
d

{'#1': [1, 3, 4, 4], '#2': [3, 1, 4, 4], '#3': [5, 5, 2, 2]}

Maximum needed size of the overall sheet is the biggest offset plus the biggest size in the same dimension:

In [7]:


max_sheet_size=max([ox+sx for (ox, oy, sx, sy) in d.values()] +
                   [oy+sy for (ox, oy, sx, sy) in d.values()])

max_sheet_size

7

And then simply (ahem) sum the arrays and count the number of elements which are greater than 1.

In [8]:
def count_overlap(string_in):
    '''
    Count overlapping squares from input of claims
    '''
    d=parse_input(string_in)
    max_sheet_size=max([ox+sx for (ox, oy, sx, sy) in d.values()] +
                       [oy+sy for (ox, oy, sx, sy) in d.values()])
    
    array_out=np.zeros((max_sheet_size, max_sheet_size), dtype=np.int)
    
    for k in d:
        array_out += claim(max_sheet_size,
                           d[k][0], d[k][1], d[k][2], d[k][3])
    
    return len([x for x in array_out.flatten() if x>=2])

In [9]:
assert count_overlap('''#1 @ 1,3: 4x4
                        #2 @ 3,1: 4x4
                        #3 @ 5,5: 2x2''') == 4

OK, the assert works. Now for my input:

In [10]:
with open('inputs/day3') as fIn:
    print(count_overlap(fIn.read()))

96569


## Part 2

Again, this is going to be a bit (!) hacky. Let's try a pairwise comparison. With luck, the arrays are handled efficiently enough that my horribly inefficient code won't be a problem :-).

Ha! That didn't work. Easier to write a function to check for overlap just from the numbers:

In [11]:
def overlap(claim1, claim2):
    '''True if the two claims overlap, fFalse otherwise'''
    [x1, y1, sx1, sy1]=[claim1[0], claim1[1], claim1[2], claim1[3]]
    [x2, y2, sx2, sy2]=[claim2[0], claim2[1], claim2[2], claim2[3]]
    
    return ((x2>=x1 and x2<x1+sx1) or (x1>=x2 and x1<x2+sx2)) and \
           ((y2>=y1 and y2<y1+sy1) or (y1>=y2 and y1<y2+sy2))

In [12]:
#Check for the original input:

d=parse_input('''#1 @ 1,3: 4x4
                        #2 @ 3,1: 4x4
                        #3 @ 5,5: 2x2''')

d['#1']

[1, 3, 4, 4]

In [13]:
overlap(d['#1'], d['#3'])

False

So we can now check see whether a given claim overlaps with any of the others:

In [14]:
# Try with claim #1:

k='#1'
[overlap(d[k1], d[k]) for k1 in d if k!=k1]

[True, False]

And we should be able to do this for all the claims:

In [15]:
{k:[overlap(d[k1], d[k]) for k1 in d if k!=k1] for k in d}

{'#1': [True, False], '#2': [True, False], '#3': [False, False]}

Add an `any` check, so it's `True` if it overlaps with any of the others:

In [16]:
coverage_dict={k:any([overlap(d[k1], d[k]) for k1 in d if k!=k1]) for k in d}
coverage_dict

{'#1': True, '#2': True, '#3': False}

And get the claims which don't have any overlap:

In [17]:
[claim for claim in coverage_dict if not coverage_dict[claim]]

['#3']

OK, works for the test case. Try with my input:

In [18]:
with open('inputs/day3') as fIn:
    d=parse_input(fIn.read())
d['#12']

[555, 514, 18, 14]

In [19]:
coverage_dict={k:any([overlap(d[k1], d[k]) for k1 in d if k!=k1]) for k in d}

[claim for claim in coverage_dict if not coverage_dict[claim]]

['#1023']